In [2]:
import os
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm

from torch.utils.data import Dataset
import torchvision.transforms as transforms
import torchvision.utils as vutils

from PIL import Image
import numpy as np
import pandas as pd

import yaml
from modules import E1, E2, Decoder, Disc
from utils import get_test_imgs, save_imgs, save_model, load_model, default_loader, CustomDataset

from firelab.config import Config

### Possible config:

In [3]:
# args = {
#         'data_path': 'glasses_train/',
#         'out': 'out_test_2x_glasses_code',
#         'lr': 0.0002,
#         'bs': 32,
#         'iters': 1250000,
#         'resize': 128,
#         'crop': 178,
#         'sep': 50,
#         'discweight': 0.001,
#         'disclr': 0.0002,
#         'progress_iter': 100,
#         'display_iter': 5000,
#         'save_iter': 10000,
#         'load': '',
#         'num_display': 12,
    
#         'device': 'cuda:0'
#         }

# with open('config.yml', 'w') as outfile:
#     yaml.dump(args, outfile)

## In general

In [ ]:
cfg = Config.load('configs/config.yml', frozen=False)

# Funcs

In [6]:
def main(args):
    
    if not os.path.exists(args.out):
        os.makedirs(args.out)

    _iter = 0

    comp_transform = transforms.Compose([
        transforms.CenterCrop(args.crop),
        transforms.Resize(args.resize),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
                                        ])

    domA_train = CustomDataset(args.data_path + 'trainA.txt', transform=comp_transform)
    domB_train = CustomDataset(args.data_path +  'trainB.txt', transform=comp_transform)

    A_label = torch.full((args.bs,), 1)
    B_label = torch.full((args.bs,), 0)
    B_separate = torch.full((args.bs, args.sep * (args.resize // 64) * (args.resize // 64)), 0)

    e1 = E1(args.sep, args.resize // 64, norm=False)
    e2 = E2(args.sep, args.resize // 64, norm=False)
    
    decoder = Decoder(args.resize // 64, norm=False)
    disc = Disc(args.sep, args.resize // 64)

    mse = nn.MSELoss()
    bce = nn.BCELoss()

    # Everything to device
    e1 = e1.to(args.device)
    e2 = e2.to(args.device)
    decoder = decoder.to(args.device)
    disc = disc.to(args.device)

    A_label = A_label.to(args.device)
    B_label = B_label.to(args.device)
    B_separate = B_separate.to(args.device)

    mse = mse.to(args.device)
    bce = bce.to(args.device)

    ae_params = list(e1.parameters()) + list(e2.parameters()) + list(decoder.parameters())
    ae_optimizer = optim.Adam(ae_params, lr=args.lr, betas=(0.5, 0.999))

    disc_params = disc.parameters()
    disc_optimizer = optim.Adam(disc_params, lr=args.disclr, betas=(0.5, 0.999))

    if args.load != '':
        save_file = os.path.join(args.load, 'checkpoint')
        _iter = load_model(save_file, e1, e2, decoder, ae_optimizer, disc, disc_optimizer)


    print('Started training...')
    while True:
        domA_loader = torch.utils.data.DataLoader(domA_train, batch_size=args.bs,
                                                  shuffle=True, num_workers=6)
        
        domB_loader = torch.utils.data.DataLoader(domB_train, batch_size=args.bs,
                                                  shuffle=True, num_workers=6)
        if _iter >= args.iters:
            break

        for domA_img, domB_img in zip(tqdm(domA_loader, leave=False), domB_loader):
            if domA_img.size(0) != args.bs or domB_img.size(0) != args.bs:
                break

            domA_img = domA_img.to(args.device)
            domB_img = domB_img.to(args.device)


            ae_optimizer.zero_grad()

            A_common = e1(domA_img)
            A_separate = e2(domA_img)
            A_encoding = torch.cat([A_common, A_separate], dim=1)

            B_common = e1(domB_img)
            B_encoding = torch.cat([B_common, B_separate], dim=1)

            A_decoding = decoder(A_encoding)
            B_decoding = decoder(B_encoding)
            
            # Reconstruction loss
            loss = mse(A_decoding, domA_img) + mse(B_decoding, domB_img)
            
            # Fooling disc
            preds_A = disc(A_common)
            preds_B = disc(B_common)
            loss += args.discweight * (bce(preds_A, B_label) + bce(preds_B, B_label))

            loss.backward(retain_graph=True)
            torch.nn.utils.clip_grad_norm_(ae_params, 5)
            ae_optimizer.step()

            # Discriminator loss
            disc_optimizer.zero_grad()

#             A_common = e1(domA_img)
#             B_common = e1(domB_img)

            disc_A = disc(A_common)
            disc_B = disc(B_common)

            loss = bce(disc_A, A_label) + bce(disc_B, B_label)

            loss.backward(retain_graph=True)
            torch.nn.utils.clip_grad_norm_(disc_params, 5)
            disc_optimizer.step()

            if _iter % args.progress_iter == 0:
                print('Outfile: %s <<>> Iteration %d' % (args.out, _iter))

            if _iter % args.display_iter == 0:
                e1 = e1.eval()
                e2 = e2.eval()
                decoder = decoder.eval()

                save_imgs(args, e1, e2, decoder, _iter)

                e1 = e1.train()
                e2 = e2.train()
                decoder = decoder.train()

            if _iter % args.save_iter == 0:
                save_file = os.path.join(args.out, 'checkpoint')
                save_model(save_file, e1, e2, decoder, ae_optimizer, disc, disc_optimizer, _iter)

            _iter += 1

In [7]:
main(cfg)

Started training...


Outfile: out_test_2x_glasses_code <<>> Iteration 0
Outfile: out_test_2x_glasses_code <<>> Iteration 100
Outfile: out_test_2x_glasses_code <<>> Iteration 200
Outfile: out_test_2x_glasses_code <<>> Iteration 300
Outfile: out_test_2x_glasses_code <<>> Iteration 400


Outfile: out_test_2x_glasses_code <<>> Iteration 500
Outfile: out_test_2x_glasses_code <<>> Iteration 600
Outfile: out_test_2x_glasses_code <<>> Iteration 700
Outfile: out_test_2x_glasses_code <<>> Iteration 800


Outfile: out_test_2x_glasses_code <<>> Iteration 900
Outfile: out_test_2x_glasses_code <<>> Iteration 1000
Outfile: out_test_2x_glasses_code <<>> Iteration 1100
Outfile: out_test_2x_glasses_code <<>> Iteration 1200


Outfile: out_test_2x_glasses_code <<>> Iteration 1300
Outfile: out_test_2x_glasses_code <<>> Iteration 1400
Outfile: out_test_2x_glasses_code <<>> Iteration 1500
Outfile: out_test_2x_glasses_code <<>> Iteration 1600


...

Outfile: out_test_2x_glasses_code <<>> Iteration 344900
Outfile: out_test_2x_glasses_code <<>> Iteration 345000
Outfile: out_test_2x_glasses_code <<>> Iteration 345100
Outfile: out_test_2x_glasses_code <<>> Iteration 345200


Outfile: out_test_2x_glasses_code <<>> Iteration 345300
Outfile: out_test_2x_glasses_code <<>> Iteration 345400
Outfile: out_test_2x_glasses_code <<>> Iteration 345500
Outfile: out_test_2x_glasses_code <<>> Iteration 345600


Outfile: out_test_2x_glasses_code <<>> Iteration 345700
Outfile: out_test_2x_glasses_code <<>> Iteration 345800
Outfile: out_test_2x_glasses_code <<>> Iteration 345900
Outfile: out_test_2x_glasses_code <<>> Iteration 346000


Outfile: out_test_2x_glasses_code <<>> Iteration 346100


KeyboardInterrupt: 